In [1]:
import pandas as pd
import numpy as np
from html import escape

In [13]:
temp = pd.DataFrame({
    'A': ['A', '<B>', 'asf'],
    '<B>': [None, None, '<asdf>'],
    '<C>': [np.nan, np.nan, '<asdf>']
}, index = ['<1>', '2', '<3>'])
temp['<C>'] = temp['<C>'].astype('category')

In [15]:
temp.style

,A,,
<1>,A,None,nan
2,,None,nan
<3>,asf,,


In [18]:
hlp.pandas_style.html_escape_dataframe(temp).style

,A,<B>,<C>
<1>,A,None,nan
2,<B>,None,nan
<3>,asf,<asdf>,<asdf>


In [22]:
from sklearn.datasets import fetch_openml
credit_g = fetch_openml('credit-g', version=1)
data = credit_g['data']
data['target'] = credit_g['target']

In [23]:
data['target'] = data['target'] == 'good'
data.loc[0:10, 'target'] = np.nan
data.loc[9:20, 'checking_status'] = np.nan
data.loc[19:30, 'credit_amount'] = np.nan

In [293]:
outer_group_by = 'checking_status'
inner_group_by = 'target'
sum_by = 'credit_amount'

In [294]:
outer_group_by = 'target'
inner_group_by = 'checking_status'
sum_by = 'credit_amount'

In [295]:
def f(x, label):
    d = {}
    d[f'{label} Count'] = x.shape[0]
    d[f'{label} Sum'] = x[sum_by].sum()
    return pd.Series(d)

In [296]:
outer_totals = data.groupby(outer_group_by).apply(f, 'Outer')
#outer_totals.columns = pd.MultiIndex.from_tuples([(outer_group_by, 'Count1'), (outer_group_by, 'Sum1')])
outer_totals = outer_totals.reset_index(level=0, drop=False)
outer_totals

,target,Outer Count,Outer Sum
0,False,296.0,1157252.0
1,True,693.0,2035914.0


In [297]:
data[sum_by].sum()

3243560.0

In [298]:
outer_totals['Outer Count Perc'] = outer_totals['Outer Count'] / data.shape[0]
outer_totals['Outer Sum Perc'] = outer_totals['Outer Sum'] / data[sum_by].sum()
outer_totals

,target,Outer Count,Outer Sum,Outer Count Perc,Outer Sum Perc
0,False,296.0,1157252.0,0.296,0.356785
1,True,693.0,2035914.0,0.693,0.627679


In [299]:
outer_totals = outer_totals[[outer_group_by, 'Outer Count', 'Outer Count Perc', 'Outer Sum', 'Outer Sum Perc']]
outer_totals

,target,Outer Count,Outer Count Perc,Outer Sum,Outer Sum Perc
0,False,296.0,0.296,1157252.0,0.356785
1,True,693.0,0.693,2035914.0,0.627679


In [300]:
inner_totals = data.groupby([outer_group_by, inner_group_by]).apply(f, 'Inner')
inner_totals = inner_totals.reset_index(level=1, drop=False)
inner_totals = inner_totals.reset_index(level=0, drop=False)
inner_totals

,target,checking_status,Inner Count,Inner Sum
0,False,<0,131.0,442342.0
1,False,0<=X<200,101.0,474190.0
2,False,>=200,14.0,24160.0
3,False,no checking,46.0,197192.0
4,True,<0,135.0,384385.0
5,True,0<=X<200,162.0,515718.0
6,True,>=200,49.0,112623.0
7,True,no checking,341.0,1009722.0


In [301]:
final = outer_totals.merge(inner_totals, on=outer_group_by, how='left')
final

,target,Outer Count,Outer Count Perc,Outer Sum,Outer Sum Perc,checking_status,Inner Count,Inner Sum
0,False,296.0,0.296,1157252.0,0.356785,<0,131.0,442342.0
1,False,296.0,0.296,1157252.0,0.356785,0<=X<200,101.0,474190.0
2,False,296.0,0.296,1157252.0,0.356785,>=200,14.0,24160.0
3,False,296.0,0.296,1157252.0,0.356785,no checking,46.0,197192.0
4,True,693.0,0.693,2035914.0,0.627679,<0,135.0,384385.0
5,True,693.0,0.693,2035914.0,0.627679,0<=X<200,162.0,515718.0
6,True,693.0,0.693,2035914.0,0.627679,>=200,49.0,112623.0
7,True,693.0,0.693,2035914.0,0.627679,no checking,341.0,1009722.0


In [302]:
final['Inner Count Perc'] = final['Inner Count'] / final['Outer Count']
final['Inner Sum Perc'] = final['Inner Sum'] / final['Outer Sum']
final

,target,Outer Count,Outer Count Perc,Outer Sum,Outer Sum Perc,checking_status,Inner Count,Inner Sum,Inner Count Perc,Inner Sum Perc
0,False,296.0,0.296,1157252.0,0.356785,<0,131.0,442342.0,0.442568,0.382235
1,False,296.0,0.296,1157252.0,0.356785,0<=X<200,101.0,474190.0,0.341216,0.409755
2,False,296.0,0.296,1157252.0,0.356785,>=200,14.0,24160.0,0.047297,0.020877
3,False,296.0,0.296,1157252.0,0.356785,no checking,46.0,197192.0,0.155405,0.170397
4,True,693.0,0.693,2035914.0,0.627679,<0,135.0,384385.0,0.194805,0.188802
5,True,693.0,0.693,2035914.0,0.627679,0<=X<200,162.0,515718.0,0.233766,0.253310
6,True,693.0,0.693,2035914.0,0.627679,>=200,49.0,112623.0,0.070707,0.055318
7,True,693.0,0.693,2035914.0,0.627679,no checking,341.0,1009722.0,0.492063,0.495955


In [303]:
new_column_order = final.columns[0:7].tolist() + [final.columns[8]] + [final.columns[7]] + [final.columns[9]]
final = final[new_column_order]

In [304]:
final

,target,Outer Count,Outer Count Perc,Outer Sum,Outer Sum Perc,checking_status,Inner Count,Inner Count Perc,Inner Sum,Inner Sum Perc
0,False,296.0,0.296,1157252.0,0.356785,<0,131.0,0.442568,442342.0,0.382235
1,False,296.0,0.296,1157252.0,0.356785,0<=X<200,101.0,0.341216,474190.0,0.409755
2,False,296.0,0.296,1157252.0,0.356785,>=200,14.0,0.047297,24160.0,0.020877
3,False,296.0,0.296,1157252.0,0.356785,no checking,46.0,0.155405,197192.0,0.170397
4,True,693.0,0.693,2035914.0,0.627679,<0,135.0,0.194805,384385.0,0.188802
5,True,693.0,0.693,2035914.0,0.627679,0<=X<200,162.0,0.233766,515718.0,0.253310
6,True,693.0,0.693,2035914.0,0.627679,>=200,49.0,0.070707,112623.0,0.055318
7,True,693.0,0.693,2035914.0,0.627679,no checking,341.0,0.492063,1009722.0,0.495955


In [305]:
outer_shifted = final[outer_group_by].shift(1)
is_first = final[outer_group_by] != outer_shifted
is_first

0     True
1    False
2    False
3    False
4     True
5    False
6    False
7    False
Name: target, dtype: bool

In [306]:
no_style_result = final
no_style_result['Outer Count'] = no_style_result['Outer Count'].where(is_first, np.nan)
no_style_result['Outer Sum'] = no_style_result['Outer Sum'].where(is_first, np.nan)
no_style_result['Outer Count Perc'] = no_style_result['Outer Count Perc'].where(is_first, np.nan)
no_style_result['Outer Sum Perc'] = no_style_result['Outer Sum Perc'].where(is_first, np.nan)
#final['Outer Sum'] = no_style_result['Outer Sum'].astype(str).where(is_first, '')
no_style_result

,target,Outer Count,Outer Count Perc,Outer Sum,Outer Sum Perc,checking_status,Inner Count,Inner Count Perc,Inner Sum,Inner Sum Perc
0,False,296.0,0.296,1157252.0,0.356785,<0,131.0,0.442568,442342.0,0.382235
1,False,NaN,NaN,NaN,NaN,0<=X<200,101.0,0.341216,474190.0,0.409755
2,False,NaN,NaN,NaN,NaN,>=200,14.0,0.047297,24160.0,0.020877
3,False,NaN,NaN,NaN,NaN,no checking,46.0,0.155405,197192.0,0.170397
4,True,693.0,0.693,2035914.0,0.627679,<0,135.0,0.194805,384385.0,0.188802
5,True,NaN,NaN,NaN,NaN,0<=X<200,162.0,0.233766,515718.0,0.253310
6,True,NaN,NaN,NaN,NaN,>=200,49.0,0.070707,112623.0,0.055318
7,True,NaN,NaN,NaN,NaN,no checking,341.0,0.492063,1009722.0,0.495955


In [307]:
new_columns = pd.MultiIndex.from_tuples([
    (outer_group_by, outer_group_by),
    (outer_group_by, 'Count'),
    (outer_group_by, 'Count Perc'),
    (outer_group_by, 'Sum'),
    (outer_group_by, 'Sum Perc'),
    (inner_group_by, inner_group_by),
    (inner_group_by, 'Count'),
    (inner_group_by, 'Count Perc'),
    (inner_group_by, 'Sum'),
    (inner_group_by, 'Sum Perc'),
])
no_style_result.columns = new_columns
no_style_result

target                                        checking_status         \
  target  Count Count Perc        Sum  Sum Perc checking_status  Count   
0  False  296.0      0.296  1157252.0  0.356785              <0  131.0   
1  False    NaN        NaN        NaN       NaN        0<=X<200  101.0   
2  False    NaN        NaN        NaN       NaN           >=200   14.0   
3  False    NaN        NaN        NaN       NaN     no checking   46.0   
4   True  693.0      0.693  2035914.0  0.627679              <0  135.0   
5   True    NaN        NaN        NaN       NaN        0<=X<200  162.0   
6   True    NaN        NaN        NaN       NaN           >=200   49.0   
7   True    NaN        NaN        NaN       NaN     no checking  341.0   

                                   
  Count Perc        Sum  Sum Perc  
0   0.442568   442342.0  0.382235  
1   0.341216   474190.0  0.409755  
2   0.047297    24160.0  0.020877  
3   0.155405   197192.0  0.170397  
4   0.194805   384385.0  0.188802  
5   0.233766   515718.0  0.253310  
6   0.070707   112623.0  0.055318  
7   0.492063  1009722.0  0.495955

style:

In [308]:
style_result = no_style_result
style_result[outer_group_by] = style_result[outer_group_by].where(is_first, np.nan)
style_result

target                                        checking_status         \
  target  Count Count Perc        Sum  Sum Perc checking_status  Count   
0  False  296.0      0.296  1157252.0  0.356785              <0  131.0   
1    NaN    NaN        NaN        NaN       NaN        0<=X<200  101.0   
2    NaN    NaN        NaN        NaN       NaN           >=200   14.0   
3    NaN    NaN        NaN        NaN       NaN     no checking   46.0   
4   True  693.0      0.693  2035914.0  0.627679              <0  135.0   
5    NaN    NaN        NaN        NaN       NaN        0<=X<200  162.0   
6    NaN    NaN        NaN        NaN       NaN           >=200   49.0   
7    NaN    NaN        NaN        NaN       NaN     no checking  341.0   

                                   
  Count Perc        Sum  Sum Perc  
0   0.442568   442342.0  0.382235  
1   0.341216   474190.0  0.409755  
2   0.047297    24160.0  0.020877  
3   0.155405   197192.0  0.170397  
4   0.194805   384385.0  0.188802  
5   0.233766   515718.0  0.253310  
6   0.070707   112623.0  0.055318  
7   0.492063  1009722.0  0.495955

In [312]:
sum_precision = 1
style_result.style.\
    format(subset=idx[:, idx[(outer_group_by, 'target')]], na_rep='').\
    format(subset=idx[:, idx[(outer_group_by, 'Count')]], precision=0, na_rep='').\
    format(subset=idx[:, idx[(outer_group_by, 'Count Perc')]], precision=4, na_rep='', formatter='{:,.2%}'.format).\
    bar(subset=idx[:, idx[(outer_group_by, 'Count Perc')]], vmin=0, vmax=1, color=hlp.color.GRAY).\
    format(subset=idx[:, idx[(outer_group_by, 'Sum')]], precision=sum_precision, na_rep='').\
    format(subset=idx[:, idx[(outer_group_by, 'Sum Perc')]], precision=4, na_rep='', formatter='{:,.2%}'.format).\
    bar(subset=idx[:, idx[(outer_group_by, 'Sum Perc')]], vmin=0, vmax=1, color=hlp.color.GRAY).\
    format(subset=idx[:, idx[(inner_group_by, 'Count')]], precision=0).\
    format(subset=idx[:, idx[(inner_group_by, 'Count Perc')]], precision=4, na_rep='', formatter='{:,.2%}'.format).\
    bar(subset=idx[:, idx[(inner_group_by, 'Count Perc')]], vmin=0, vmax=1, color=hlp.color.GRAY).\
    format(subset=idx[:, idx[(inner_group_by, 'Sum')]], precision=sum_precision).\
    format(subset=idx[:, idx[(inner_group_by, 'Sum Perc')]], precision=4, formatter='{:,.2%}'.format).\
    bar(subset=idx[:, idx[(inner_group_by, 'Sum Perc')]], vmin=0, vmax=1, color=hlp.color.GRAY)

In [261]:
sum_precision = 1
style_result.style.\
    format(subset=idx[:, idx[('target', 'target')]], na_rep='').\
    format(subset=idx[:, idx[('target', 'Count')]], precision=0, na_rep='').\
    bar(subset=idx[:, idx[('target', 'Count')]], vmin=0, vmax=data.shape[0], color=hlp.color.GRAY).\
    format(subset=idx[:, idx[('target', 'Count Perc')]], precision=4, na_rep='', formatter='{:,.2%}'.format).\
    bar(subset=idx[:, idx[('target', 'Count Perc')]], vmin=0, vmax=1, color=hlp.color.GRAY).\
    format(subset=idx[:, idx[('target', 'Sum')]], precision=sum_precision, na_rep='').\
    bar(subset=idx[:, idx[('target', 'Sum')]], vmin=0, vmax=data[sum_by].sum(), color=hlp.color.GRAY).\
    format(subset=idx[:, idx[('target', 'Sum Perc')]], precision=4, na_rep='', formatter='{:,.2%}'.format).\
    bar(subset=idx[:, idx[('target', 'Sum Perc')]], vmin=0, vmax=1, color=hlp.color.GRAY)

In [157]:
final[outer_group_by] = final[outer_group_by].astype(str).where(is_first, '')
final['Outer Count'] = final['Outer Count'].astype(str).where(is_first, '')
final['Outer Sum'] = final['Outer Sum'].astype(str).where(is_first, '')
final

,target,Outer Count,Outer Sum,checking_status,Inner Count,Inner Sum,Perc,Sum
0,False,296.0,1157252.0,<0,131.0,442342.0,0.442568,0.382235
1,,,,0<=X<200,101.0,474190.0,0.341216,0.409755
2,,,,>=200,14.0,24160.0,0.047297,0.020877
3,,,,no checking,46.0,197192.0,0.155405,0.170397
4,True,693.0,2035914.0,<0,135.0,384385.0,0.194805,0.188802
5,,,,0<=X<200,162.0,515718.0,0.233766,0.253310
6,,,,>=200,49.0,112623.0,0.070707,0.055318
7,,,,no checking,341.0,1009722.0,0.492063,0.495955


In [163]:
final = final[['target', 'Outer Count', 'checking_status', 'Inner Count', 'Perc', 'Outer Sum', 'Inner Sum', 'Sum']]
final

,target,Outer Count,checking_status,Inner Count,Perc,Outer Sum,Inner Sum,Sum
0,False,296.0,<0,131.0,0.442568,1157252.0,442342.0,0.382235
1,,,0<=X<200,101.0,0.341216,,474190.0,0.409755
2,,,>=200,14.0,0.047297,,24160.0,0.020877
3,,,no checking,46.0,0.155405,,197192.0,0.170397
4,True,693.0,<0,135.0,0.194805,2035914.0,384385.0,0.188802
5,,,0<=X<200,162.0,0.233766,,515718.0,0.253310
6,,,>=200,49.0,0.070707,,112623.0,0.055318
7,,,no checking,341.0,0.492063,,1009722.0,0.495955


In [158]:
final.style.bar(vmin=0)

,target,Outer Count,Outer Sum,checking_status,Inner Count,Inner Sum,Perc,Sum
0,False,296.0,1157252.0,<0,131.000000,442342.000000,0.442568,0.382235
1,,,,0<=X<200,101.000000,474190.000000,0.341216,0.409755
2,,,,>=200,14.000000,24160.000000,0.047297,0.020877
3,,,,no checking,46.000000,197192.000000,0.155405,0.170397
4,True,693.0,2035914.0,<0,135.000000,384385.000000,0.194805,0.188802
5,,,,0<=X<200,162.000000,515718.000000,0.233766,0.253310
6,,,,>=200,49.000000,112623.000000,0.070707,0.055318
7,,,,no checking,341.000000,1009722.000000,0.492063,0.495955
